In [1]:
import dill

In [7]:
data = dill.load(
    open("/Users/chanb/research/ualberta/icl/scratch/results/synthetic-ic_predictor/p_high_0.5-dataset_size_64-p_relevant_context_0.5-seed_0-09-15-24_20_58_25-4d5603f5-2e4c-451a-ba6d-3dcf2da3a9df/models/10000.dill", "rb")
)

In [8]:
data

{'model': {'ic_predictor': {'params': {'log_temp': Array(0., dtype=float32)}}},
 'opt_state': (InjectStatefulHyperparamsState(count=Array(10000, dtype=int32), hyperparams={'b1': Array(0.9, dtype=float32), 'b2': Array(0.999, dtype=float32), 'eps': Array(1.e-08, dtype=float32), 'eps_root': Array(0., dtype=float32), 'learning_rate': Array(0.0003, dtype=float32)}, hyperparams_states={'learning_rate': WrappedScheduleState(count=Array(10000, dtype=int32))}, inner_state=(ScaleByAdamState(count=Array(10000, dtype=int32), mu={'ic_predictor': {'params': {'log_temp': Array(0., dtype=float32)}}}, nu={'ic_predictor': {'params': {'log_temp': Array(0., dtype=float32)}}}), EmptyState())),)}